## Gun Barrel Diagram

**TODO**
1. make plot interactive --> may help interp
2. column data source: name, status, operator, length

### Import libraries

In [2]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import Label
from bokeh.layouts import gridplot
from bokeh.io import output_notebook

import lasio
import pandas as pd

import math

### Enter values from data sources

In [3]:
# measured from QGIS
lease_width = [0, 5280]
tvd_range = [10000, 9100] # inverted depth
half_sec = [lease_width[1]/2, lease_width[1]/2]

# tops pulled from spreadsheet
bone_spring_cosmo = [6253, 6253]
wfmp_a_cosmo = [9214,9214]
wfmp_b_cosmo = 9659

print(wfmp_b_cosmo - wfmp_a_cosmo[0])

# TVD at LP (INC = 90*) and TVD from header
strat_state_unit_4WA = [[2260, 9503], [2260, 9566]] # WFMP A DUC
strat_state_unit_23WB = [[1920, 9685], [1920, 9711]] # WFMP B DUC

# TVD at LP (INC = 90*) and TVD from header
stoneman_state_3927A_2H = [[4505, 9383],[4505, 9483]] # WFMP A ACTIVE
stoneman_state_3927B_1H = [[4890, 9760], [4890, 9849]] # WFMP B ACTIVE

# future Laterals
stoneman_state_3927A_6H = [2960, 9500] # WFMP A PERMIT
stoneman_state_3927B_5H = [2960, 9850] # WFMP B PERMIT
stoneman_state_3927A_4H = [3485, 9500] # WFMP A PERMIT
stoneman_state_3927B_3H = [4010, 9850] # WFMP B PERMIT

445


### Calculate spacing and hypotenuse distance

In [4]:

stone_2H = stoneman_state_3927A_2H[1]
stone_1H = stoneman_state_3927B_1H[1]

stone_hyp = math.sqrt((stone_2H[0]-stone_1H[0])**2 + (stone_2H[1]-stone_1H[1])**2)
stone_x = stone_1H[0]-stoneman_state_3927B_3H[0]

strat_4wa = strat_state_unit_4WA[1]
strat_23b = strat_state_unit_23WB[1]

strat_hyp = math.sqrt((strat_4wa[0]-strat_23b[0])**2 + (strat_4wa[1]-strat_23b[1])**2)

print(f"stone_hyp: {stone_hyp:.2f}")
print(f"stone_x: {stone_x:.2f}")
print(f"strat_hyp: {strat_hyp:.2f}")

stone_hyp: 531.21
stone_x: 880.00
strat_hyp: 369.63


### Load and process Cosmo Investments 45-45 1V type log

In [5]:
# cosmo investments 45-45 1V ECP las 
cosmo_path = "key_logs/42-389-33719-00-00_ECP_RS0031583884.las"
cosmo_las = lasio.read(cosmo_path)

# With log step in well header calulate a smoothing window with factor of 10
window = int(1 / cosmo_las.well.STEP.value)*10
print(f"smoothing window length: {window}")

# convert las object to dataframe
cosmo_df = cosmo_las.df()

# select only GR_RSE and reset index
cosmo_df = cosmo_df[["GR_RSE"]].reset_index()

# smooth GR_RSE with rolling mean
cosmo_df['GR_MEAN'] = cosmo_df['GR_RSE'].rolling(window, center=True).mean()
cosmo_df.describe()

smoothing window length: 20


,DEPT,GR_RSE,GR_MEAN
count,3534.000000,3457.000000,3438.000000
mean,10183.250000,86.736742,86.624855
std,510.161127,26.509012,17.412225
min,9300.000000,12.707000,24.467000
25%,9741.625000,73.901000,77.589988
50%,10183.250000,88.881000,86.210075
75%,10624.875000,102.168000,95.310288
max,11066.500000,270.720000,141.543750


### Plot gun barrel

In [6]:
# initiate gun barrel plot
p1 = figure(plot_width=900, plot_height=500, x_axis_label="Sec. 39 (5280ft)", x_range=lease_width, y_axis_label="TVD (FT)", y_range=tvd_range)

# set title
p1.title.text = "Stoneman/Strategery State Unit: WFMP Gun Barrel"
p1.title.align = "center"
p1.title.text_font_size = "16pt"

# style elements
p1.xgrid.visible = False
p1.ygrid.visible = True

# add half section line
p1.line(half_sec, tvd_range, line_color="dimgray", width=1)
p1.add_layout(Label(x=2640, y=9990, text='1/2 Sec.', text_font_size="10pt", text_color="black", angle=math.radians(90)))

# add and label FM tops for WFMP A/B
p1.line(lease_width, wfmp_a_cosmo, line_color="black", width=2, line_dash="dashed")
p1.add_layout(Label(x=10, y=wfmp_a_cosmo[0]-2, text='WFMP A', text_font_size="10pt", text_color="black"))
p1.line(lease_width, 9659, line_color="black", width=2, line_dash="dashed")
p1.add_layout(Label(x=10, y=9659-2, text='WFMP B', text_font_size="10pt", text_color="black"))

# add DUCs
# p1.circle(strat_state_unit_4WA[0][0], strat_state_unit_4WA[0][1], line_color="red", fill_color="white", width=2, size=12)
# p1.circle(strat_state_unit_23WB[0][0], strat_state_unit_23WB[0][1], line_color="blue", fill_color="white", width=2, size=12)
p1.circle(strat_state_unit_4WA[1][0], strat_state_unit_4WA[1][1], line_color="red", fill_color="white", width=2, size=12)
p1.circle(strat_state_unit_23WB[1][0], strat_state_unit_23WB[1][1], line_color="blue", fill_color="white", width=2, size=12)

# add ACTIVEs
# p1.circle(stoneman_state_3927A_2H[0][0], stoneman_state_3927A_2H[0][1], line_color="red", fill_color="red", width=2, size=12)
# p1.circle(stoneman_state_3927B_1H[0][0], stoneman_state_3927B_1H[0][1], line_color="blue", fill_color="blue", width=2, size=12)
p1.circle(stoneman_state_3927A_2H[1][0], stoneman_state_3927A_2H[1][1], line_color="red", fill_color="red", width=2, size=12)
p1.circle(stoneman_state_3927B_1H[1][0], stoneman_state_3927B_1H[1][1], line_color="blue", fill_color="blue", width=2, size=12)

# add PERMITs
p1.square(stoneman_state_3927A_6H[0], stoneman_state_3927A_6H[1], line_color="red", fill_color="white", width=2, size=10, angle=math.radians(45))
p1.square(stoneman_state_3927B_5H[0], stoneman_state_3927B_5H[1], line_color="blue", fill_color="white", width=2, size=10, angle=math.radians(45))
p1.square(stoneman_state_3927A_4H[0], stoneman_state_3927A_4H[1], line_color="red", fill_color="white", width=2, size=10, angle=math.radians(45))
p1.square(stoneman_state_3927B_3H[0], stoneman_state_3927B_3H[1], line_color="blue", fill_color="white", width=2, size=10, angle=math.radians(45))

GlyphRenderer(id='1091', ...)

### Plot type log

In [7]:
# initiate type log plot
p2 = figure(plot_width=200, plot_height=500, x_axis_label="GR_MEAN (API)", x_range=[0,210], y_axis_label="TVD (FT)", y_range=tvd_range)
# p2.line("GR_RSE","DEPT", source=cosmo_df, line_color="dimgray", line_width=0.75)
p2.line("GR_MEAN", "DEPT", source=cosmo_df, line_color="black", line_width=2)

# set title
p2.title.text = "Cosmo Investments 45-45 1V"
p2.title.align = "center"
p2.title.text_font_size = "10pt"

# style elements
p2.xgrid.visible = False
p2.ygrid.visible = True
p2.yaxis.visible = False

p2.line(lease_width, wfmp_a_cosmo, line_color="black", width=2, line_dash="dashed")
p2.add_layout(Label(x=10, y=wfmp_a_cosmo[0]-2, text='WFMP A', text_font_size="10pt", text_color="black"))
p2.line(lease_width, 9659, line_color="black", width=2, line_dash="dashed")
p2.add_layout(Label(x=10, y=9659-2, text='WFMP B', text_font_size="10pt", text_color="black"))

### Render gun barrel and type log in a grid

In [8]:
# display in notebook
output_notebook()

# save static HTML
# output_file("gun_barrel.html")

# assemble grid and show
grid = gridplot([p1, p2], ncols=2)
show(grid)

Loading BokehJS ...